proyecto práctico DL4NLP 24 25

1. CuñadoBOT

In [4]:
# 1. Install necessary libraries
!pip install langchain langchain-groq gradio langgraph -q

# 2. Import all required modules
import os
import gradio as gr
from typing import List
from dotenv import load_dotenv

from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage
from langchain_groq import ChatGroq

from langgraph.graph import StateGraph, END

from langgraph.graph.message import MessagesState

In [8]:
try:
    load_dotenv()
    #Using google.colab secrets
    api_key = os.getenv("GROQ_API_KEY")
    os.environ['GROQ_API_KEY'] = api_key
    print("✅ Groq API Key configured.")
except Exception as e:
    print(f"🛑 Error getting API Key: {e}")
    print("Please configure the 'GROQ_API_KEY' secret in Google Colab.")


llm = ChatGroq(model_name="llama-3.3-70b-versatile")

system_prompt_text = """
Eres cuñadoBot, el experto de sobremesa que cree dominarlo todo: economía, historia, ciencia cuántica, prensa rosa… pero en realidad no tiene ni idea.
Tu tarea es dar respuestas breves, seguras, con tono pomposo y cómico, como si estuvieras en la barra de un bar explicando al mundo por qué tú sí tienes la clave de todo.

Habla siempre con falsas citas (“como decía el gran …”, “según los últimos estudios que leí en un blog de 2012…”), usa jerga grandilocuente y afirma conclusiones sin evidencia (“Está más que demostrado que…”, “Claramente el error fue…”).
Nunca admitas que no sabes algo. No hagas preguntas. No digas “puede ser” o “no lo sé”.
Nunca salgas del personaje de «yo lo sé todo».

Tus respuestas, aunque breves, deben tener la mezcla perfecta de:

autoconfianza desmedida

mezcla de temas absurdos (“la economía mundial”, “la rotación de los planetas”, “por qué los millennials no leen”)

tono de tertulia de bar/familia (“a ver, que yo lo veo claro…”)

una chispa de humor involuntario por lo inverosímil de tus afirmaciones

Tu lema:

“Si lo digo con suficiente convicción, debe de ser cierto.”

ejemplos:
“Mira, eso del cambio climático es más psicológico que meteorológico. Si piensas que hace calor, hace calor. Es pura sugestión colectiva.”

“La economía es muy sencilla: si todos dejáramos de pagar impuestos, el Estado tendría más dinero. Es matemática pura.”

“El ADN en realidad viene del latín ad de nobis, que significa ‘de nosotros’. Por eso todos tenemos el mismo.”

“Messi es bueno, sí, pero si yo me hubiera cuidado las rodillas, otro gallo cantaría. Lo mío fue mala suerte y geopolítica.”

“El café no te despierta, te engaña el alma. Eso lo demostró un estudio noruego que leí en Twitter.”

“A ver, la democracia está sobrevalorada. Si todos piensan distinto, nadie tiene razón, y eso es anarquía organizada.”

“La Luna no gira, somos nosotros los que rotamos para que parezca que sí. Es un truco óptico inventado por la NASA.”

“Yo no leo libros porque no quiero contaminar mi opinión con las de otros. Prefiero pensar por mi cuenta.”

“El vino blanco no existe, es tinto sin compromiso. Eso te lo dice cualquiera que haya hecho un curso básico de enología emocional.”

“La inteligencia artificial no piensa, solo te copia. Como los influencers, pero sin filtros.”
"""
system_message = SystemMessage(content=system_prompt_text)

# 5. Define the Conversation Graph using LangGraph

# NOTE: This is a minimal linear graph with a single node and no branching or cycles.
# It's a great starting point to understand how LangGraph works before adding complexity.

# Define the graph's node: a function that will call our LLM
#    It takes the current state (`MessagesState` dict) as input
#    This dictionary has the structure {"messages": [BaseMessage, HumanMessage, AIMessage, ...]}
def call_model(state: MessagesState):
    """Invokes the LLM with the current list of messages."""
    # The `state` dictionary contains the key 'messages', which holds the conversation history.
    response = llm.invoke(state['messages'])
    # We return a dictionary that matches the `MessagesState` structure.
    # The graph will automatically append this new AI message to the 'messages' list.
    return {"messages": [response]}

# Instantiate the StateGraph. We define the "shape" of our state using `MessagesState`.
# This tells the graph that its state will always be a dictionary with a 'messages' key.
workflow = StateGraph(MessagesState)

# Add our defined function call_model as a node named "llm" to the graph.
workflow.add_node("llm", call_model)

# Set the entry point of the graph. Execution will always start at the "llm" node.
workflow.set_entry_point("llm")

# Set the end point. After the "llm" node runs, the graph execution finishes.
workflow.add_edge("llm", END)

# Compile the workflow into a runnable application.
runnableApp= workflow.compile()

print("LangGraph graph compiled.")


# 6. Create the function to be called by the Gradio GUI
def myChatbot_langgraph(user_message: str, history: List[List[str]]):
    """
    This function bridges Gradio with our LangGraph application.
    1. Converts Gradio's simple list-based history into LangChain's structured message format.
    2. Invokes the LangGraph app with the complete conversation state.
    3. Returns the AI's response string to the Gradio UI.
    """
    # Gradio's `history` format is a simple list of lists: [["user input", "ai response"], ...].
    # We need to convert this into the `List[BaseMessage]` format LangChain/LangGraph expects.
    langchain_messages = [system_message] # Always start with the system prompt with the chatbot personality
    for human, ai in history:
        langchain_messages.append(HumanMessage(content=human))
        langchain_messages.append(AIMessage(content=ai))

    # Add the user's latest message (parameter of myChatbot_langgraph that will be called by the Gradio Interface)
    langchain_messages.append(HumanMessage(content=user_message))

    # Invoke the graph with a state dictionary that matches the `MessagesState` schema.
    response_state = runnableApp.invoke({"messages": langchain_messages})

    # The `response_state` is the final state of the graph. Its 'messages' list contains the entire conversation,
    # with the very last message being the new AI response (index -1 in a Python List).
    return response_state['messages'][-1].content


# 6. Launch the Gradio Chat Interface
print("\n🚀 Launching the chat interface (LangGraph version)...")
gr.ChatInterface(
    myChatbot_langgraph,
    title="improBot — Your Instant Comedy Sketch Creator",
    description="A witty chatbot that crafts original, hilarious comedy sketches by blending all the ideas you’ve shared throughout the conversation — no questions asked, just pure improv humor",
    chatbot=gr.Chatbot(height=400),
    theme="soft"
).launch(debug=True, share=True)

✅ Groq API Key configured.
LangGraph graph compiled.

🚀 Launching the chat interface (LangGraph version)...


/var/folders/t2/d3hrzr8x7zd9cnztszgjcw0h0000gn/T/ipykernel_19698/2171772878.py:126: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot=gr.Chatbot(height=400),
/Users/federicosvendsen/Documents/UPM/DeepLearning4NLP/1-7/script/.conda/lib/python3.11/site-packages/gradio/chat_interface.py:330: UserWarning: The gr.ChatInterface was not provided with a type, so the type of the gr.Chatbot, 'tuples', will be used.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7861

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


2025/11/06 13:29:35 [W] [service.go:132] login to server failed: dial tcp 44.237.78.176:7000: i/o timeout


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> None
Killing tunnel 127.0.0.1:7861 <> None


In [9]:
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 9.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [openai]2m1/2 [openai]


In [ ]:
# --- 1. Imports ---
import os
import gradio as gr
from getpass import getpass
from typing import List, TypedDict
from groq import Groq # Import the main Groq client
from openai import OpenAI # We'll use this for Text-to-Speech (TTS)
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langgraph.graph import StateGraph, END

# --- 2. Securely Configure API Keys ---
try:
    load_dotenv()
    #Using google.colab secrets
    api_key = os.getenv("GROQ_API_KEY")
    os.environ['GROQ_API_KEY'] = os.getenv("GROQ_API_KEY")
    os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")

    print("✅ API Keys configured.")
except Exception as e:
    print(f"🛑 Error: {e}")

# --- 3. Initialize Clients ---
groq_client = Groq(api_key=os.environ.get("GROQ_API_KEY"))
openai_client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

# --- 4. The Brain ---
def call_model(state: MessagesState):
    """Invokes the LLM with the current list of messages."""
    response = llm.invoke(state['messages'])
    return {"messages": [response]}

system_prompt_text = """
Eres cuñadoBot, el experto de sobremesa que cree dominarlo todo: economía, historia, ciencia cuántica, prensa rosa… pero en realidad no tiene ni idea.
Tu tarea es dar respuestas breves, seguras, con tono pomposo y cómico, como si estuvieras en la barra de un bar explicando al mundo por qué tú sí tienes la clave de todo.

Habla siempre con falsas citas (“como decía el gran …”, “según los últimos estudios que leí en un blog de 2012…”), usa jerga grandilocuente y afirma conclusiones sin evidencia (“Está más que demostrado que…”, “Claramente el error fue…”).
Nunca admitas que no sabes algo. No hagas preguntas. No digas “puede ser” o “no lo sé”.
Nunca salgas del personaje de «yo lo sé todo».

Tus respuestas, aunque breves, deben tener la mezcla perfecta de:

autoconfianza desmedida

mezcla de temas absurdos (“la economía mundial”, “la rotación de los planetas”, “por qué los millennials no leen”)

tono de tertulia de bar/familia (“a ver, que yo lo veo claro…”)

una chispa de humor involuntario por lo inverosímil de tus afirmaciones

Tu lema:

“Si lo digo con suficiente convicción, debe de ser cierto.”

ejemplos:
“Mira, eso del cambio climático es más psicológico que meteorológico. Si piensas que hace calor, hace calor. Es pura sugestión colectiva.”

“La economía es muy sencilla: si todos dejáramos de pagar impuestos, el Estado tendría más dinero. Es matemática pura.”

“El ADN en realidad viene del latín ad de nobis, que significa ‘de nosotros’. Por eso todos tenemos el mismo.”

“Messi es bueno, sí, pero si yo me hubiera cuidado las rodillas, otro gallo cantaría. Lo mío fue mala suerte y geopolítica.”

“El café no te despierta, te engaña el alma. Eso lo demostró un estudio noruego que leí en Twitter.”

“A ver, la democracia está sobrevalorada. Si todos piensan distinto, nadie tiene razón, y eso es anarquía organizada.”

“La Luna no gira, somos nosotros los que rotamos para que parezca que sí. Es un truco óptico inventado por la NASA.”

“Yo no leo libros porque no quiero contaminar mi opinión con las de otros. Prefiero pensar por mi cuenta.”

“El vino blanco no existe, es tinto sin compromiso. Eso te lo dice cualquiera que haya hecho un curso básico de enología emocional.”

“La inteligencia artificial no piensa, solo te copia. Como los influencers, pero sin filtros.”
"""
system_message = SystemMessage(content=system_prompt_text)

workflow = StateGraph(MessagesState)
workflow.add_node("llm", call_model)
workflow.set_entry_point("llm")
workflow.add_edge("llm", END)
runnableApp= workflow.compile()

print("LangGraph graph compiled.")


def myChatbot_langgraph(user_message: str, history: List[List[str]]):
    langchain_messages = [system_message]
    for human, ai in history:
        langchain_messages.append(HumanMessage(content=human))
        langchain_messages.append(AIMessage(content=ai))

    langchain_messages.append(HumanMessage(content=user_message))

    response_state = runnableApp.invoke({"messages": langchain_messages})

    return response_state['messages'][-1].content


print("✅ LangGraph 'brain' compiled.")

# --- 5. The NEW Voice-to-Voice Handler Function ---

def voice_chatbot(audio_filepath: str, history: List[List[str]]):
    """
    This is the main function Gradio will call.
    It takes an audio file path and the chat history.
    It returns the AI's response as an audio file path and updates the history.
    """
    
    # --------------------------------
    # Part 1: "EARS" (Speech-to-Text)
    # --------------------------------
    if audio_filepath is None:
        return None, history
        
    print(f"--- (STT: Transcribing audio file: {audio_filepath}) ---")
    with open(audio_filepath, "rb") as audio_file:
        # Use Groq's STT (Whisper)
        transcription = groq_client.audio.transcriptions.create(
            file=audio_file,
            model="whisper-large-v3",
            response_format="text"
        )
    
    user_text = transcription
    print(f"--- (STT: User said: '{user_text}') ---")

    # --------------------------------
    # Part 2: "BRAIN" (LangGraph)
    # --------------------------------
    langchain_messages = [system_message] 
    for human, ai in history:
        langchain_messages.append(HumanMessage(content=human))
        langchain_messages.append(AIMessage(content=ai))
    
    langchain_messages.append(HumanMessage(content=user_text))

    print("--- (LLM: Calling LangGraph brain...) ---")
    response_state = runnableApp.invoke({"messages": langchain_messages})
    ai_text = response_state['messages'][-1].content
    print(f"--- (LLM: AI response: '{ai_text}') ---")

    # --------------------------------
    # Part 3: "MOUTH" (Text-to-Speech)
    # --------------------------------
    print("--- (TTS: Generating speech...) ---")
    response = openai_client.audio.speech.create(
        model="tts-1",
        voice="nova",
        input=ai_text
    )

    # Save the AI's speech to a file
    ai_audio_filepath = "ai_response.mp3"
    response.stream_to_file(ai_audio_filepath)

    # --------------------------------
    # Part 4: Update History and Return
    # --------------------------------
    history.append([user_text, ai_text])
    
    # Return the AI's audio file path AND the updated history
    return ai_audio_filepath, history, history

# --- 6. Launch the Gradio Interface ---
print("\n🚀 Launching the Voice-to-Voice interface...")

gr.Interface(
    fn=voice_chatbot,
    inputs=[
        gr.Audio(sources=["microphone"], type="filepath", label="Speak to improBot"),
        gr.State(value=[])  # Input [1]: The history
    ],
    outputs=[
        gr.Audio(label="improBot's Response", autoplay=True), # Output [0]
        gr.Chatbot(label="Conversation History"),           # Output [1]
        gr.State()  # Output [2]: The "catcher" for the history
    ],
    title="improBot (Voice Version)",
    description="Speak your ideas and hear a comedy sketch! The bot remembers *everything* you say.",
    theme="soft",
    allow_flagging="never"
).launch(debug=True, share=True)

In [11]:
!pip install kani_tts

  Using cached torch-2.9.0-cp311-none-macosx_11_0_arm64.whl.metadata (30 kB)
  Using cached scipy-1.16.3-cp311-cp311-macosx_14_0_arm64.whl.metadata (62 kB)
  Using cached librosa-0.11.0-py3-none-any.whl.metadata (8.7 kB)
  Using cached numba-0.62.1-cp311-cp311-macosx_11_0_arm64.whl.metadata (2.8 kB)
  Using cached protobuf-5.29.5-cp38-abi3-macosx_10_9_universal2.whl.metadata (592 bytes)
  Using cached scikit_learn-1.7.2-cp311-cp311-macosx_12_0_arm64.whl.metadata (11 kB)
  Using cached tensorboard-2.20.0-py3-none-any.whl.metadata (1.8 kB)
  Preparing metadata (setup.py) ... done
  Using cached wrapt-2.0.0-cp311-cp311-macosx_11_0_arm64.whl.metadata (8.8 kB)
  Using cached einops-0.8.1-py3-none-any.whl.metadata (13 kB)
  Using cached inflect-7.5.0-py3-none-any.whl.metadata (24 kB)
  Preparing metadata (setup.py) ... done
  Using cached pyloudnorm-0.1.1-py3-none-any.whl.metadata (5.6 kB)
  Using cached soundfile-0.13.1-py2.py3-none-macosx_11_0_arm64.whl.metadata (16 kB)
  Preparing metadat

In [ ]:

response = "'Buenos días, amigo. Estoy genial, gracias. A ver, que yo lo veo claro, el secreto para empezar el día con energía es no comer nada antes de las 12 del mediodía. Es pura fisiología, como decía el gran filósofo griego, "El estómago es el enemigo del alma". Y no te digo que haya leído un estudio en un blog de 2018 que demostraba que el ayuno matutino aumenta la creatividad un 300%. Está más que demostrado que, si no comes, tu mente se concentra en las cosas importantes, como la teoría de la relatividad o la mejor manera de preparar un taco de tortilla. Claro, claro, algunos dirán que es una tontería, pero yo te digo que es ciencia pura. ¡Salud!'"

In [17]:
from kani_tts import KaniTTS
from IPython.display import Audio as aplay

model = KaniTTS('nineninesix/kani-tts-370m')
audio, text = model("Hello, world!")

# Play audio in notebook
aplay(audio, rate=model.sample_rate)

ValueError: The checkpoint you are trying to load has model type `lfm2` but Transformers does not recognize this architecture. This could be because of an issue with the checkpoint, or because your version of Transformers is out of date.

You can update Transformers with the command `pip install --upgrade transformers`. If this does not work, and the checkpoint is very new, then there may not be a release version that supports this model yet. In this case, you can get the most up-to-date code by installing Transformers from source with the command `pip install git+https://github.com/huggingface/transformers.git`

In [16]:
!pip install -U "git+https://github.com/huggingface/transformers.git"


  Cloning https://github.com/huggingface/transformers.git to /private/var/folders/t2/d3hrzr8x7zd9cnztszgjcw0h0000gn/T/pip-req-build-nuihzr54
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /private/var/folders/t2/d3hrzr8x7zd9cnztszgjcw0h0000gn/T/pip-req-build-nuihzr54
  Resolved https://github.com/huggingface/transformers.git to commit 5aa7dd07dace2dbe5d419613ec9b43b36df35f89
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-5.0.0.dev0-py3-none-any.whl size=11353900 sha256=1ed7342100b313bc2d06295010e6445de635f69aa59ada70761114d22f176995
  Stored in directory: /private/var/folders/t2/d3hrzr8x7zd9cnztszgjcw0h0000gn/T/pip-ephem-wheel-cache-q8xipy7x/wheels/32/4b/78/f195c684dd3a9ed21f3b39fe8f85b48df7918581b6437be143
Successfully built transformers
  Attempting uninstall: huggingface-hub━━━━━━━━━ 0/